In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import torch.nn as nn
import torch.nn.functional as F

from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device, device.type)

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())

cuda cuda
Files already downloaded and verified
Files already downloaded and verified


In [3]:
%%bash
nvidia-smi

Sun Dec  8 15:06:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A1000 6GB Lap...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              4W /   45W |    1360MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
class BatchNormConv2d(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size=3, stride=1, padding=1, groups=1):
        super(BatchNormConv2d, self).__init__()
        self.gelu = nn.GELU()
        self.conv = nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=kernel_size, stride=stride, padding=padding)
        self.bn = nn.BatchNorm2d(out_ch)
        
    def forward(self, x):
        return self.gelu(self.bn(self.conv(x)))

In [5]:
class InvertedBottleneckBlock(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch, ks=3, stride=1):
        super(InvertedBottleneckBlock, self).__init__()

        self.expand = BatchNormConv2d(in_ch, mid_ch, kernel_size=1, stride=stride, padding=0)
        if ks < 3:
            raise ValueError("The kernel for the depthwise convolution needs to be at least 3x3")
        #the groups should be the input channel here 'mid_ch' when using bigger sized kernels
        self.padding = 0 if ks % 2 == 0 else (ks // 2) #handle asymmetric padding since the division works for uneven number here
        self.maintain = BatchNormConv2d(mid_ch, mid_ch, kernel_size=ks, groups=mid_ch, padding=self.padding)
        self.reduce = BatchNormConv2d(mid_ch, out_ch, kernel_size=1, stride=stride, padding=0)
        
        self.residual = None
        #added dimension projection layer for differeing in and out sizes
        if in_ch != out_ch:
            self.residual = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=stride)
    def forward(self, x):
        res = self.residual(x) if self.residual else x
        #print("start", x.shape)
        x = self.expand(x)
        #print("after expand", x.shape)
        x = self.maintain(x)
        #print("after maintain", x.shape)
        x = self.reduce(x)
        #print("after reduce", x.shape)
        if x.shape != res.shape:
            raise RuntimeError(f"Shape mismatch: x={x.shape}, res={res.shape}")
        return F.gelu(x + res)

In [6]:
class InvertedBottleneckModel(nn.Module):
    def __init__(self, in_ch=3, num_classes=10, bottleneck_configs=None):
        super(InvertedBottleneckModel, self).__init__()
        if bottleneck_configs is None:
            bottleneck_configs = [
                (3, 16, 32, 3),
                (32, 64, 128, 5),
                (128, 256, 512, 3),
            ]

        self.blocks = nn.ModuleList([InvertedBottleneckBlock(*config) for config in bottleneck_configs])
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.mlp = nn.Linear(bottleneck_configs[-1][2]*4*4, num_classes)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
            x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.mlp(x)
        
        return F.softmax(x, dim=1)

In [9]:
conv_model = InvertedBottleneckModel()
if device.type == "cuda":
    conv_model = conv_model.to('cuda')
learning_rate = 0.0001
BATCH_SIZE = 128
epochs = 20

train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, pin_memory=True)

#move dataset loader to gpu for each batch all at once to avoid moving memory during training
#train_loader = move_dataloader_to_device(train_loader, device)
#test_loader = move_dataloader_to_device(test_loader, device)

opt = torch.optim.Adam(conv_model.parameters(), lr=learning_rate)
print(conv_model, opt, device)

InvertedBottleneckModel(
  (blocks): ModuleList(
    (0): InvertedBottleneckBlock(
      (expand): BatchNormConv2d(
        (gelu): GELU(approximate='none')
        (conv): Conv2d(3, 16, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (maintain): BatchNormConv2d(
        (gelu): GELU(approximate='none')
        (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (reduce): BatchNormConv2d(
        (gelu): GELU(approximate='none')
        (conv): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (residual): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): InvertedBottleneckBlock(
      (expand): BatchNormConv2d(
        (gelu): GELU(approximate='

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    avg_loss = fit_one_cycle(conv_model, train_loader, opt, epoch, writer, device, flatten=False)
    acc = prediction_accuracy(conv_model, test_loader, device, flatten=False)
    print(f"Accuracy: {acc * 100}% ({acc})\nLoss: {avg_loss}")

In [10]:
#testing multiple configs
model_configs = [
    [(3, 16, 32, 3), (32, 64, 128, 5), (128, 256, 512, 3)],  # Default config
    [(3, 16, 32, 3), (32, 32, 64, 3), (64, 128, 128, 3)],    # Smaller mid-channels
    [(3, 32, 64, 5), (64, 128, 256, 5), (256, 512, 1024, 5)], # Larger kernel sizes
]

best_accuracy = 0
best_config = None

epochs = 20

for idx, bottleneck_config in enumerate(model_configs):
    print(f"Training configuration {idx + 1}/{len(model_configs)}: {bottleneck_config}")
    
    conv_model = InvertedBottleneckModel(in_ch=3, num_classes=10, bottleneck_configs=bottleneck_config).to(device)
    if device.type == "cuda":
        conv_model = conv_model.to('cuda')
    
    opt = torch.optim.Adam(conv_model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    BATCH_SIZE = 128

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/inverted_bottleneck_trainer_{}_{}'.format(idx, timestamp))

    acc = 0
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}")
        avg_loss = fit_one_cycle(conv_model, train_loader, opt, epoch, writer, device, flatten=False)
        acc = prediction_accuracy(conv_model, test_loader, device, flatten=False)
        print(f"Accuracy: {acc * 100}% ({acc})\nLoss: {avg_loss}")

    conv_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = conv_model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    
    accuracy = correct / total
    print(f"Configuration {idx + 1}: Accuracy = {accuracy:.4f}")
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_config = bottleneck_config

print("\nBest Configuration:", best_config)
print("Best Accuracy:", best_accuracy)


Training configuration 1/3: [(3, 16, 32, 3), (32, 64, 128, 5), (128, 256, 512, 3)]
Epoch 1



raining: 100%|███████████████████████████████| 391/391 [00:13<00:00, 28.49it/s]

Correct: 4260, Total: 10000, Accuracy: 0.43
Accuracy: 42.6% (0.426)
Loss: 2.044842086340252
Epoch 2



raining: 100%|███████████████████████████████| 391/391 [00:13<00:00, 29.43it/s]

Correct: 4190, Total: 10000, Accuracy: 0.42
Accuracy: 41.9% (0.419)
Loss: 2.011384650280601
Epoch 3


...

Correct: 3743, Total: 10000, Accuracy: 0.37
Accuracy: 37.43% (0.3743)
Loss: 2.0774422319311845
Epoch 19



raining: 100%|███████████████████████████████| 391/391 [00:37<00:00, 10.49it/s]

Correct: 3350, Total: 10000, Accuracy: 0.34
Accuracy: 33.5% (0.335)
Loss: 2.0966429208454334
Epoch 20



raining: 100%|███████████████████████████████| 391/391 [00:37<00:00, 10.48it/s]

Correct: 4200, Total: 10000, Accuracy: 0.42
Accuracy: 42.0% (0.42)
Loss: 2.0372430901778373
Configuration 3: Accuracy = 0.4243

Best Configuration: [(3, 16, 32, 3), (32, 32, 64, 3), (64, 128, 128, 3)]
Best Accuracy: 0.7769


In [13]:
best_config = [(3, 16, 32, 3), (32, 32, 64, 3), (64, 128, 128, 3)]
conv_model = InvertedBottleneckModel(in_ch=3, num_classes=10, bottleneck_configs=best_config).to(device)

opt = torch.optim.Adam(conv_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
BATCH_SIZE = 128
epochs = 30

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/inverted_bottleneck_trainer_validation_{}'.format(timestamp))
training_and_validation_loop(conv_model, train_loader, test_loader, epochs, writer, device, opt, timestamp, flatten=False)

Epoch 1



raining: 100%|███████████████████████████████| 391/391 [00:10<00:00, 38.65it/s]

Correct: 4950, Total: 10000, Accuracy: 0.49
Accuracy: 49.5% (0.495)
Loss: 1.993074655532837



alidation: 100%|███████████████████████████████| 79/79 [00:01<00:00, 66.87it/s]

LOSS train 1.993074655532837 valid 1.940560221672058
Saving model
Epoch 2


...

LOSS train 1.5306029821697034 valid 1.6658766269683838
Epoch 30



raining: 100%|███████████████████████████████| 391/391 [00:09<00:00, 40.94it/s]

Correct: 7715, Total: 10000, Accuracy: 0.77
Accuracy: 77.14999999999999% (0.7715)
Loss: 1.5216111070231388


Validation: 100%|███████████████████████████████| 79/79 [00:01<00:00, 76.51it/s]

LOSS train 1.5216111070231388 valid 1.6646318435668945
Saving model


In [20]:
from torchinfo import summary
print(summary(conv_model, (128, 3, 32, 32)))

Layer (type:depth-idx)                   Output Shape              Param #
InvertedBottleneckModel                  [128, 10]                 --
├─ModuleList: 1-5                        --                        (recursive)
│    └─InvertedBottleneckBlock: 2-1      [128, 32, 32, 32]         --
│    │    └─Conv2d: 3-1                  [128, 32, 32, 32]         128
│    │    └─BatchNormConv2d: 3-2         [128, 16, 32, 32]         96
│    │    └─BatchNormConv2d: 3-3         [128, 16, 32, 32]         2,352
│    │    └─BatchNormConv2d: 3-4         [128, 32, 32, 32]         608
├─MaxPool2d: 1-2                         [128, 32, 16, 16]         --
├─ModuleList: 1-5                        --                        (recursive)
│    └─InvertedBottleneckBlock: 2-2      [128, 64, 16, 16]         --
│    │    └─Conv2d: 3-5                  [128, 64, 16, 16]         2,112
│    │    └─BatchNormConv2d: 3-6         [128, 32, 16, 16]         1,120
│    │    └─BatchNormConv2d: 3-7         [128, 32, 16, 1

In [21]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 16037), started 1:05:03 ago. (Use '!kill 16037' to kill it.)